<h2> Estimasi PI dengan Monte Carlo </h2>


Berikut adalah koding yang digunkana untuk melakukan estimasi nilai PI menggunakan metode monte carlo. Metode Monte Carlo untuk estimasi nilai PI bekerja dengan mencari rasio antara titik data yang berada dalam sebuah lingkaran dengan titik data yang berada di luar lingkaran. Semakin banyak titik data yang dibangkitkan secara acak maka estimasi nilai PI semakin akurat.
<br>
Pernjelasan Video: https://www.youtube.com/watch?v=ELetCV_wX_c


<h2> File Mapper </h2>


Berikut adalah code pada mapper yang digunakan untuk melakukan transformasi data.
"(x1, y1) (x2, y2) (x3, y3) menjadi (didalam) (diluar) (didalam)"


In [45]:
%%file piMapper.py
#!/home/ubuntu/anaconda3/bin/python

#Pastikan shebang di atas mengarah ke python interpreter anda.
import sys
"""
Analisa kode berikut untuk menambahkan pemahaman anda!
"""
for value in sys.stdin:
    x , y = list(map(float, value.split()))
    result = 1 if x*x + y*y <= 1 else 0
    print(result)

Overwriting piMapper.py


<h2> File Reducer. </h2>


Berikut adalah code pada mapper yang digunakan untuk menghitung estimasi PI menggunakan metode monte carlo.
(didalam) (diluar) (didalam) menjadi (estimasi PI) <br><br>
Rumus:<br>
4 * jumlah_titik_dalam_lingkaran / jumlah_data_keseluruhan

In [46]:
%%file piReducer.py
#!/home/ubuntu/anaconda3/bin/python

#Pastikan shebang di atas mengarah ke python interpreter anda.
import sys
"""
Analisa kode berikut untuk menambahkan pemahaman anda!
"""
current_sum = 0
count = 0
for value in sys.stdin:
    current_sum += int(value)
    count +=1
print(4 * current_sum / count)


Overwriting piReducer.py


<h2>Berikan izin untuk eksekusi file</h2>


In [47]:
!chmod u+x piMapper.py piReducer.py

<h2> Bangkitkan data secara acak </h2>


In [3]:
#Kode berikut digunakan untuk membangkitkan 10000000 angka acak dari -1 hingga 1
#angka yang dihasilkan berupa koordinat <x, y>
import random
count = 10000000
with open("random.txt", "w") as numbers:
    for i in range(count):
        numbers.write("%f %f\n" % (random.uniform(-1,1), random.uniform(-1,1)))

<h2> Menjalankan program tanpa Hadoop </h2>


In [49]:
!cat random.txt | python piMapper.py  | python piReducer.py

3.13868


<h2> Persiapan menjakan dengan Hadoop </h2>
-bersihkan input sebelumnya<br>
-bersihkan output sebelumnya<br>
-buat direktori input<br>
-unggah input ke hdfs<br>

In [52]:
!hdfs dfs -rm -r input/pi
!hdfs dfs -rm -r output/pi
!hdfs dfs -mkdir -p input/pi
!hdfs dfs -put random.txt input/pi

Deleted input/pi
rm: `output/pi': No such file or directory


<h2> Jalankan program menggunakan Hadoop MapReduce </h2>


In [53]:
!hadoop jar /home/ubuntu/hadoop-2.8.5/share/hadoop/tools/lib/hadoop-streaming-2.8.5.jar \
-mapper piMapper.py \
-reducer piReducer.py \
-input input/pi/random.txt \
-output output/pi

19/06/29 09:00:49 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
19/06/29 09:00:49 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
19/06/29 09:00:49 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
19/06/29 09:00:49 INFO mapred.FileInputFormat: Total input files to process : 1
19/06/29 09:00:49 INFO mapreduce.JobSubmitter: number of splits:1
19/06/29 09:00:50 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1571049563_0001
19/06/29 09:00:50 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
19/06/29 09:00:50 INFO mapreduce.Job: Running job: job_local1571049563_0001
19/06/29 09:00:50 INFO mapred.LocalJobRunner: OutputCommitter set in config null
19/06/29 09:00:50 INFO mapred.LocalJobRunner: OutputCommitter is org.apache.hadoop.mapred.FileOutputCommitter
19/06/29 09:00:50 INFO output.FileOutputCommitter: Fil

19/06/29 09:00:52 INFO reduce.MergeManagerImpl: Merged 1 segments, 500002 bytes to disk to satisfy reduce memory limit
19/06/29 09:00:52 INFO reduce.MergeManagerImpl: Merging 1 files, 500006 bytes from disk
19/06/29 09:00:52 INFO reduce.MergeManagerImpl: Merging 0 segments, 0 bytes from memory into reduce
19/06/29 09:00:52 INFO mapred.Merger: Merging 1 sorted segments
19/06/29 09:00:52 INFO mapred.Merger: Down to the last merge-pass, with 1 segments left of total size: 499998 bytes
19/06/29 09:00:52 INFO mapred.LocalJobRunner: 1 / 1 copied.
19/06/29 09:00:52 INFO streaming.PipeMapRed: PipeMapRed exec [/home/ubuntu/notebook/pi/./piReducer.py]
19/06/29 09:00:52 INFO Configuration.deprecation: mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
19/06/29 09:00:52 INFO Configuration.deprecation: mapred.map.tasks is deprecated. Instead, use mapreduce.job.maps
19/06/29 09:00:52 INFO streaming.PipeMapRed: R/W/S=1/0/0 in:NA [rec/s] out:NA [rec/s]
19/06/29 09:00:52 INFO s

<h2> Tampilkan output </h2>


In [54]:
!hdfs dfs -cat output/pi/*

3.13868	
